## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
0,0,Cabo San Lucas,MX,22.8909,-109.9124,94.60,61,0,11.50,clear sky
1,1,Saldanha,ZA,-33.0117,17.9442,46.45,83,55,7.18,broken clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,41.11,87,75,0.00,broken clouds
3,3,Narsaq,GL,60.9167,-46.0500,52.02,86,95,3.27,overcast clouds
4,4,Oga,IT,46.4631,10.3470,66.88,82,68,3.20,broken clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 20
What is the maximum temperature you would like for your trip? 80


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather Description
1,1,Saldanha,ZA,-33.0117,17.9442,46.45,83,55,7.18,broken clouds
2,2,Punta Arenas,CL,-53.1500,-70.9167,41.11,87,75,0.00,broken clouds
3,3,Narsaq,GL,60.9167,-46.0500,52.02,86,95,3.27,overcast clouds
4,4,Oga,IT,46.4631,10.3470,66.88,82,68,3.20,broken clouds
5,5,Longjiang,CN,47.3303,123.1836,66.29,88,6,9.78,clear sky
...,...,...,...,...,...,...,...,...,...,...
697,697,Narasannapeta,IN,18.4167,84.0500,79.20,81,100,3.67,overcast clouds
699,699,Sundsvall,SE,62.3913,17.3063,63.97,80,100,2.30,overcast clouds
700,700,Kimbe,PG,-5.5502,150.1429,79.47,78,54,3.15,broken clouds
701,701,The Pas,CA,53.8251,-101.2541,66.22,72,100,13.80,overcast clouds


In [39]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                513
City                   513
Country                513
Lat                    513
Lng                    513
Max Temp               513
Humidity               513
Cloudiness             513
Wind Speed             513
Weather Description    513
dtype: int64

In [40]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                513
City                   513
Country                513
Lat                    513
Lng                    513
Max Temp               513
Humidity               513
Cloudiness             513
Wind Speed             513
Weather Description    513
dtype: int64

In [41]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Saldanha,ZA,46.45,broken clouds,-33.0117,17.9442,
2,Punta Arenas,CL,41.11,broken clouds,-53.1500,-70.9167,
3,Narsaq,GL,52.02,overcast clouds,60.9167,-46.0500,
4,Oga,IT,66.88,broken clouds,46.4631,10.3470,
5,Longjiang,CN,66.29,clear sky,47.3303,123.1836,
6,East London,ZA,64.31,overcast clouds,-33.0153,27.9116,
7,Sao Joao Da Barra,BR,75.29,scattered clouds,-21.6403,-41.0511,
8,Tabuk,PH,73.40,overcast clouds,17.4189,121.4443,
9,Chicama,PE,61.34,scattered clouds,-7.8447,-79.1469,
11,Bambous Virieux,MU,70.05,few clouds,-20.3428,57.7575,


In [52]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key,
}

# Iterate through the dataframe
for index, row in hotel_df.iterrows():

    # Get lat and lng
    lat = row['Lat']
    lng = row['Lng']
    
    params["location"] = f'{lat},{lng}'
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make API request
    hotels = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(f"Hotel found! Adding to data.")
    
    except:
        print(f"No hotel in radius ... skipping.")
        
    print("----------------")        

Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
No hotel in radius ... skipping.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
-

Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
No hotel in radius ... skipping.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
No hotel in radius ... skipping.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to dat

Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
No hotel in radius ... skipping.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
No hotel in radius ... skipping.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to data.
----------------
Hotel found! Adding to dat

In [54]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("",np.nan,inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
1,Saldanha,ZA,46.45,broken clouds,-33.0117,17.9442,Saldanha Bay Hotel
2,Punta Arenas,CL,41.11,broken clouds,-53.1500,-70.9167,Hotel Cabo de Hornos
3,Narsaq,GL,52.02,overcast clouds,60.9167,-46.0500,Igdlo Guesthouse
4,Oga,IT,66.88,broken clouds,46.4631,10.3470,Alpen
5,Longjiang,CN,66.29,clear sky,47.3303,123.1836,Longjiang Hotel


In [55]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [56]:
# 9. Add information to the info box.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

,Lat,Lng
1,-33.0117,17.9442
2,-53.1500,-70.9167
3,60.9167,-46.0500
4,46.4631,10.3470
5,47.3303,123.1836
...,...,...
697,18.4167,84.0500
699,62.3913,17.3063
700,-5.5502,150.1429
701,53.8251,-101.2541
